In [1]:
import pandas as pd
import pandashelpers

In [2]:
metadata = pd.read_csv('data/metadata.csv')

In [3]:
strength = pd.read_csv('data/3dprinter/data.csv')

In [ ]:
controllable = ["print_speed",
             "layer_height",
             "infill_density",
             "nozzle_temperature",
             "wall_thickness"
            ]

input_args = ['--solid-infill-speed',
                                           '--layer-height',
                                           '--fill-density',
                                           '--temperature',
                                           '--solid-layers'
                                           ]

In [31]:
strength_summary = strength.deduplicate(controllable,
                     numeric_agg='mean',
                     non_numeric_agg='first'
                    ).reset_index(drop=True)

In [51]:
metadata.merge(strength_summary,
               left_on=["--solid-infill-speed",
                       "--layer-height",
                       "--fill-density",
                        "--temperature"
                       ],
               right_on=["print_speed",
                         "layer_height",
                         "infill_density",
                         "nozzle_temperature"
                        ],
               how='outer').filter(regex='temp',axis='columns')

,--temperature,bed_temperature,first_layer_bed_temperature,first_layer_temperature,standby_temperature_delta,temperature,nozzle_temperature,bed_temperature
0,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN
1,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN
2,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN
3,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN
4,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN
5,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN
6,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN
7,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN
8,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN
9,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN


In [48]:
#metadata['--temperature']
#strength_summary['nozzle_temperature']

0     False
1     False
2      True
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13     True
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29     True
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38     True
39    False
40    False
41    False
42    False
43    False
44     True
45    False
46    False
47    False
48    False
49    False
Name: nozzle_temperature, dtype: bool